In [ ]:
!pip install transformers datasets

     |████████████████████████████████| 3.4 MB 5.0 MB/s 
     |████████████████████████████████| 306 kB 48.4 MB/s 
     |████████████████████████████████| 3.3 MB 42.5 MB/s 
     |████████████████████████████████| 895 kB 46.3 MB/s 
     |████████████████████████████████| 596 kB 44.9 MB/s 
     |████████████████████████████████| 61 kB 441 kB/s 
     |████████████████████████████████| 243 kB 47.0 MB/s 
     |████████████████████████████████| 132 kB 49.8 MB/s 
     |████████████████████████████████| 1.1 MB 42.1 MB/s 
     |████████████████████████████████| 271 kB 34.4 MB/s 
     |████████████████████████████████| 160 kB 42.1 MB/s 
     |████████████████████████████████| 192 kB 17.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
import os
from google.colab import drive
import datasets
import pyarrow as pa
import pyarrow.dataset as ds
import pandas as pd
import torch
from torch.utils.data import Dataset
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer
from transformers import TrainingArguments, Trainer

In [ ]:
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Colab Notebooks/Kaggle') #切換該目錄
os.listdir() #確認目錄內容

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


['kaggle_test_dataset.csv',
 'kaggle_train_dataset.csv',
 'kaggle_test_submission.csv',
 'cache_dir',
 'results']

In [ ]:
input_train = pd.read_csv("kaggle_train_dataset.csv", delimiter = "\t")
input_test = pd.read_csv("kaggle_test_dataset.csv", delimiter = "\t")

In [ ]:
# Train
train_dataset = input_train[['review', 'rating']]

# empty_review = ((train_dataset['review'].isnull()) | (train_dataset['review'] == ' '))
# train_dataset = train_dataset[~empty_review]
# train_dataset = train_dataset[['drugName','condition','review','date','usefulCount','rating']]
train_dataset['review'].str.replace('"','')
train_dataset['review'] = train_dataset['review'].str[:500]

# Test
test = input_test

empty_review = ((test['review'].isnull()) | (test['review'] == ' '))
test = test[~empty_review]
test = test[['drugName','condition','review','date','usefulCount',]]
test['review'].str.replace('"','')
test['review'] = test['review'].str[:100]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
train_dataset.rename(columns={'rating': 'label'}, inplace=True)
train_dataset = train_dataset.astype({"label": int})
train_dataset['label'] = train_dataset['label']-1

In [ ]:
train = datasets.Dataset.from_pandas(train_dataset)
train = train.remove_columns('__index_level_0__')
train_dict = train.train_test_split(test_size=0.3)

In [ ]:
type(train_dict['train']['label'][0])

int

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [ ]:
def preprocess_function(examples):
    return tokenizer(examples["review"], padding=True, truncation=True)

tokenized_train = train_dict.map(preprocess_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
tokenized_train

DatasetDict({
    train: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'review'],
        num_rows: 3500
    })
    test: Dataset({
        features: ['attention_mask', 'input_ids', 'label', 'review'],
        num_rows: 1500
    })
})

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=10)

Downloading:   0%|          | 0.00/256M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.bias', 'pre_classifi

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train['train'],
    eval_dataset=tokenized_train['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: review.
***** Running training *****
  Num examples = 3500
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1314


Step,Training Loss
500,1.831000
1000,1.382600


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json


Step,Training Loss
500,1.831000
1000,1.382600




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1314, training_loss=1.5010896912085592, metrics={'train_runtime': 9233.9559, 'train_samples_per_second': 2.274, 'train_steps_per_second': 0.142, 'total_flos': 461794445924400.0, 'train_loss': 1.5010896912085592, 'epoch': 6.0})

In [ ]:
trainer.save_model('fine-tune-v1')

Saving model checkpoint to fine-tune-v1.json
Configuration saved in fine-tune-v1.json/config.json
Model weights saved in fine-tune-v1.json/pytorch_model.bin
tokenizer config file saved in fine-tune-v1.json/tokenizer_config.json
Special tokens file saved in fine-tune-v1.json/special_tokens_map.json


In [ ]:
test_dataset = input_test[['review']]

empty_review = ((test_dataset['review'].isnull()) | (test_dataset['review'] == ' '))
test_dataset['review'].str.replace('"','')
test_dataset['review'] = test_dataset['review'].str[:200]

test = datasets.Dataset.from_pandas(test_dataset)
tokenized_test = test.map(preprocess_function, batched=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


  0%|          | 0/50 [00:00<?, ?ba/s]

In [ ]:
tokenized_test

Dataset({
    features: ['attention_mask', 'input_ids', 'review'],
    num_rows: 50000
})

In [ ]:
prediction = trainer.predict(tokenized_test)

The following columns in the test set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: review.
***** Running Prediction *****
  Num examples = 50000
  Batch size = 16


In [ ]:
prediction_label = pd.DataFrame(prediction.label_ids)

In [ ]:
submit = pd.read_csv("kaggle_test_submission.csv", delimiter = "\t")

